In [1]:
import os
import pandas as pd

HOME = os.path.expanduser('~')
df = pd.read_csv(HOME + '/temp/labeledTrainData.tsv', header=0, sep='\t', quoting=3)
df

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."
...,...,...,...
24995,"""3453_3""",0,"""It seems like more consideration has gone int..."
24996,"""5064_1""",0,"""I don't believe they made this film. Complete..."
24997,"""10905_3""",0,"""Guy is a loser. Can't get girls, needs to bui..."
24998,"""10194_3""",0,"""This 30 minute documentary Buñuel made in the..."


In [2]:
import re

df['review'] = df['review'].str.replace('<br />', ' ')
df['review'] = df['review'].apply(lambda x: re.sub("[^a-zA-Z]", " ", x))

In [3]:
from sklearn.model_selection import train_test_split

target = df['sentiment']
feature = df.drop(['id', 'sentiment'], axis=1, inplace=False)
xtr, xte, ytr, yte = train_test_split(feature, target, test_size=0.3, random_state=156)
print(xtr.shape, xte.shape, ytr.shape, yte.shape)

(17500, 1) (7500, 1) (17500,) (7500,)


In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipe = Pipeline([
    ('cv', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr', LogisticRegression(C=10))
])

pipe.fit(xtr['review'], ytr)
pred = pipe.predict(xte['review'])
prob = pipe.predict_proba(xte['review'])[:, 1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.
      format(accuracy_score(yte ,pred), roc_auc_score(yte, prob)))

/home/arti/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


예측 정확도는 0.8860, ROC-AUC는 0.9503


In [5]:
pipe1 = Pipeline([
    ('tv', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr', LogisticRegression(C=10))
])

pipe1.fit(xtr['review'], ytr)
pred1 = pipe1.predict(xte['review'])
prob1 = pipe1.predict_proba(xte['review'])[:, 1]

print(accuracy_score(yte, pred1), roc_auc_score(yte, prob1))

0.8936 0.959799823582973


In [6]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/arti/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /home/arti/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/arti/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /home/arti/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/arti/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/arti/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to
[nltk_data]    |     /home/arti/nltk_data

[nltk_data]    |   Package timit is already up-to-date!
[nltk_data]    | Downloading package toolbox to
[nltk_data]    |     /home/arti/nltk_data...
[nltk_data]    |   Package toolbox is already up-to-date!
[nltk_data]    | Downloading package treebank to
[nltk_data]    |     /home/arti/nltk_data...
[nltk_data]    |   Package treebank is already up-to-date!
[nltk_data]    | Downloading package twitter_samples to
[nltk_data]    |     /home/arti/nltk_data...
[nltk_data]    |   Package twitter_samples is already up-to-date!
[nltk_data]    | Downloading package udhr to /home/arti/nltk_data...
[nltk_data]    |   Package udhr is already up-to-date!
[nltk_data]    | Downloading package udhr2 to /home/arti/nltk_data...
[nltk_data]    |   Package udhr2 is already up-to-date!
[nltk_data]    | Downloading package unicode_samples to
[nltk_data]    |     /home/arti/nltk_data...
[nltk_data]    |   Package unicode_samples is already up-to-date!
[nltk_data]    | Downloading package universal_treebanks

True

In [7]:
from nltk.corpus import wordnet as wn

term = 'present'

synsets = wn.synsets(term)
print(type(synsets), len(synsets), synsets)

<class 'list'> 18 [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


In [8]:
for synset in synsets:
    print('##### Synset name: ', synset.name(), ' #####')
    print('POS: ', synset.lexname())
    print('Definition: ', synset.definition())
    print('Lemmas: ', synset.lemma_names())

##### Synset name:  present.n.01  #####
POS:  noun.time
Definition:  the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas:  ['present', 'nowadays']
##### Synset name:  present.n.02  #####
POS:  noun.possession
Definition:  something presented as a gift
Lemmas:  ['present']
##### Synset name:  present.n.03  #####
POS:  noun.communication
Definition:  a verb tense that expresses actions or states at the time of speaking
Lemmas:  ['present', 'present_tense']
##### Synset name:  show.v.01  #####
POS:  verb.perception
Definition:  give an exhibition of to an interested audience
Lemmas:  ['show', 'demo', 'exhibit', 'present', 'demonstrate']
##### Synset name:  present.v.02  #####
POS:  verb.communication
Definition:  bring forward and present to the mind
Lemmas:  ['present', 'represent', 'lay_out']
##### Synset name:  stage.v.01  #####
POS:  verb.creation
Definition:  perform (a play), especially on a stage
Lemmas:  ['stage', 'present

In [9]:
tree = wn.synset('tree.n.01')
lion = wn.synset('lion.n.01')
tiger = wn.synset('tiger.n.02')
cat = wn.synset('cat.n.01')
dog = wn.synset('dog.n.01')

entities = [tree, lion, tiger, cat, dog]
similarities = []
entity_names = [entity.name().split('.')[0] for entity in entities]

for entity in entities:
    similarity = [round(entity.path_similarity(compared_entity), 2)
                 for compared_entity in entities]
    similarities.append(similarity)
    
df1 = pd.DataFrame(similarities, columns=entity_names, index=entity_names)
df1

,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


In [10]:
from nltk.corpus import sentiwordnet as swn

senti = list(swn.senti_synsets('slow'))
print(type(senti), len(senti), senti)

<class 'list'> 11 [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


In [11]:
father = swn.senti_synset('father.n.01')
print(father.pos_score(), father.neg_score(), father.obj_score())

0.0 0.0 1.0


In [12]:
fabulous = swn.senti_synset('fabulous.a.01')
print(fabulous.pos_score(), fabulous.neg_score(), fabulous.obj_score())

0.875 0.125 0.0


In [13]:
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB

In [14]:
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag

def polarity(text):
    sentiment = 0.0
    tokens = 0

    lemmatizer = WordNetLemmatizer()
    raws = sent_tokenize(text)

    for raw in raws:
        tagged = pos_tag(word_tokenize(raw))
        
        for word, tag in tagged:

            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue

            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue

            synsets = wn.synsets(lemma,pos=wn_tag)
            if not synsets:
                continue

            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())
            tokens += 1

    if not tokens:
        return 0

    if sentiment >= 0:
        return 1

    return 0

In [15]:
df['preds'] = df['review'].apply(lambda x: polarity(x))
yta = df['sentiment'].values
preds = df['preds'].values

In [16]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print(confusion_matrix(yta, preds))
print('정확도:',accuracy_score(yta, preds))
print('정밀도:',precision_score(yta, preds))
print('재현율:',recall_score(yta, preds))

[[7668 4832]
 [3636 8864]]
정확도: 0.66128
정밀도: 0.647196261682243
재현율: 0.70912


In [17]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti = SentimentIntensityAnalyzer()
score = senti.polarity_scores(df['review'][0])
print(senti)

In [18]:
def vader_polarity(review, threshold=0.1):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    agg_score = scores['compound']
    return 1 if agg_score >= threshold else 0 

In [19]:
df['vader_preds'] = df['review'].apply(lambda x: vader_polarity(x, 0.1))
yta = df['sentiment'].values
preds = df['vader_preds'].values

print(confusion_matrix(yta, preds))
print('정확도:',accuracy_score(yta, preds))
print('정밀도:',precision_score(yta, preds))
print('재현율:',recall_score(yta, preds))

[[ 6736  5764]
 [ 1867 10633]]
정확도: 0.69476
정밀도: 0.6484722815149113
재현율: 0.85064
